In [1]:
import os
from subprocess import check_output
import yaml

In [2]:
with open('packages.yml') as package_file:
    packages = yaml.load(package_file)

In [3]:
CWD = os.path.abspath(os.path.curdir)

In [4]:
cmd = ['git', 'log', '--oneline', '--since="2017-11-10"']
ncommits = len(check_output(cmd).splitlines())

In [5]:
activity = {}
total_commits = 0
for package in packages:
    subpackages = packages[package].split()
    for subpackage in subpackages:
        os.chdir(CWD)
        os.chdir('tmp/{subpackage}'.format(subpackage=subpackage))
        ncommits = len(check_output(cmd).splitlines())
        total_commits += ncommits
        activity[subpackage] = ncommits
       

In [6]:
activity, total_commits

({'libpysal': 159,
  'esda': 9,
  'pointpats': 15,
  'giddy': 84,
  'mapclassify': 2,
  'splot': 163,
  'gwr': 103,
  'spglm': 20,
  'spint': 21,
  'spreg': 30,
  'spvcm': 20},
 626)